# Day 9: Movie Theater

## 1. Section: Define the Puzzle Constraints
The puzzle asks us to find the largest rectangle formed by two red tiles (given coordinates) as opposite corners.
- **Part 1**: The rectangle can be anywhere. Area = `(abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)`.
- **Part 2**: The rectangle must be contained within the polygon defined by the sequence of red tiles (connected by green tiles). The polygon includes its boundary and interior.

## 2. Section: Represent the Puzzle State
We represent the puzzle state as a list of `(x, y)` coordinates. For Part 2, the order of these coordinates matters as they define the polygon boundary.


In [ ]:
def read_input(file_path):
    points = []
    with open(file_path, 'r') as f:
        for line in f:
            x, y = map(int, line.strip().split(','))
            points.append((x, y))
    return points

def solve_part1(points):
    max_area = 0
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            x1, y1 = points[i]
            x2, y2 = points[j]
            area = (abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)
            if area > max_area:
                max_area = area
    return max_area

# Test with test.txt
test_points = read_input('test.txt')
print(f"Test Part 1: {solve_part1(test_points)}")

# Solve with input.txt
input_points = read_input('input.txt')
print(f"Part 1: {solve_part1(input_points)}")


## 3. Section: Implement the Solver Algorithm
We implement two solvers:
- `solve_part1`: Iterates all pairs and finds max area.
- `solve_part2`: Iterates all pairs, checks if the rectangle is valid (inside polygon), and finds max area. We optimize by sorting pairs by area descending.


### Geometric Helper Functions

To solve Part 2, we need to determine if a candidate rectangle is completely inside the polygon defined by the red tiles.
A rectangle is inside the polygon if:
1.  All its vertices are inside or on the boundary of the polygon.
2.  No edge of the polygon intersects the *interior* of the rectangle.

We implement the following helper functions:
-   `is_on_segment(px, py, p1, p2)`: Checks if point `(px, py)` lies on the line segment `p1-p2`.
-   `is_point_inside_or_on_boundary(px, py, edges)`: Uses the **Ray Casting Algorithm** to check if a point is inside the polygon. We cast a ray to the right (`y = py + 0.5`) and count intersections with vertical edges. An odd number of intersections means the point is inside. We also explicitly check if the point is on the boundary.
-   `rect_intersects_edges(x_min, x_max, y_min, y_max, edges)`: Checks if any edge of the polygon passes through the interior of the rectangle. This is necessary because a non-convex polygon could "cut into" the rectangle even if all rectangle corners are inside.


In [ ]:
def is_on_segment(px, py, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if x1 == x2: # Vertical
        return x1 == px and min(y1, y2) <= py <= max(y1, y2)
    else: # Horizontal
        return y1 == py and min(x1, x2) <= px <= max(x1, x2)

def is_point_inside_or_on_boundary(px, py, edges):
    # Check if on boundary
    for p1, p2 in edges:
        if is_on_segment(px, py, p1, p2):
            return True
            
    # Ray casting (to the right)
    # Use y + 0.5 to avoid vertices
    intersections = 0
    for p1, p2 in edges:
        x1, y1 = p1
        x2, y2 = p2
        
        # Check vertical edges
        if x1 == x2:
            # Edge is vertical at x1
            # Check if ray crosses this edge
            # Ray is at y = py + 0.5
            # Edge y range is [min(y1, y2), max(y1, y2)]
            # We need x1 > px
            if x1 > px:
                if min(y1, y2) <= py + 0.5 <= max(y1, y2):
                    intersections += 1
                    
    return intersections % 2 == 1

def rect_intersects_edges(x_min, x_max, y_min, y_max, edges):
    # Check if any edge intersects the OPEN rectangle (x_min, x_max) x (y_min, y_max)
    for p1, p2 in edges:
        x1, y1 = p1
        x2, y2 = p2
        
        if x1 == x2: # Vertical edge
            # Check if x1 is in (x_min, x_max)
            if x_min < x1 < x_max:
                # Check if y range overlaps with (y_min, y_max)
                # Overlap of [y_a, y_b] and (y_min, y_max)
                y_a, y_b = min(y1, y2), max(y1, y2)
                if max(y_a, y_min) < min(y_b, y_max):
                    return True
        else: # Horizontal edge
            # Check if y1 is in (y_min, y_max)
            if y_min < y1 < y_max:
                # Check if x range overlaps with (x_min, x_max)
                x_a, x_b = min(x1, x2), max(x1, x2)
                if max(x_a, x_min) < min(x_b, x_max):
                    return True
    return False


### Part 2 Solver Logic

The `solve_part2` function iterates through all pairs of red tiles to form candidate rectangles.
To optimize the search:
1.  We calculate the area for all pairs first.
2.  We sort the pairs by area in descending order.
3.  We check the validity of each rectangle starting from the largest.
4.  The first valid rectangle we find is guaranteed to be the largest one.

For each candidate rectangle defined by opposite corners `(x1, y1)` and `(x2, y2)`:
-   We determine the other two corners `(x_min, y_max)` and `(x_max, y_min)`.
-   We check if these two corners are inside or on the boundary of the polygon.
-   We check if any polygon edge intersects the interior of the rectangle.
-   If all checks pass, we return the area.


In [ ]:
def solve_part2(points):
    # Build edges
    edges = []
    for i in range(len(points)):
        edges.append((points[i], points[(i + 1) % len(points)]))
        
    # Generate all pairs and sort by area descending
    pairs = []
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            x1, y1 = points[i]
            x2, y2 = points[j]
            area = (abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)
            pairs.append((area, points[i], points[j]))
            
    pairs.sort(key=lambda x: x[0], reverse=True)
    
    for area, p1, p2 in pairs:
        x1, y1 = p1
        x2, y2 = p2
        x_min, x_max = min(x1, x2), max(x1, x2)
        y_min, y_max = min(y1, y2), max(y1, y2)
        
        # Check the other two corners
        c1 = (x_min, y_max)
        c2 = (x_max, y_min)
        
        # Optimization: Check if corners are inside first
        if not is_point_inside_or_on_boundary(c1[0], c1[1], edges):
            continue
        if not is_point_inside_or_on_boundary(c2[0], c2[1], edges):
            continue
            
        # Check if any edge intersects the interior
        if rect_intersects_edges(x_min, x_max, y_min, y_max, edges):
            continue
            
        return area
        
    return 0

print(f"Test Part 2: {solve_part2(test_points)}")
print(f"Part 2: {solve_part2(input_points)}")


## 4. Section: Visualize the Solution

To verify the result and provide more insight, we can modify the solver to return the specific points that form the largest rectangle.
We define `solve_part2_with_details` to return the area and the two opposite corner points.


In [ ]:
def solve_part2_with_details(points):
    edges = []
    for i in range(len(points)):
        edges.append((points[i], points[(i + 1) % len(points)]))
        
    pairs = []
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            x1, y1 = points[i]
            x2, y2 = points[j]
            area = (abs(x1 - x2) + 1) * (abs(y1 - y2) + 1)
            pairs.append((area, points[i], points[j]))
            
    pairs.sort(key=lambda x: x[0], reverse=True)
    
    for area, p1, p2 in pairs:
        x1, y1 = p1
        x2, y2 = p2
        x_min, x_max = min(x1, x2), max(x1, x2)
        y_min, y_max = min(y1, y2), max(y1, y2)
        
        c1 = (x_min, y_max)
        c2 = (x_max, y_min)
        
        if not is_point_inside_or_on_boundary(c1[0], c1[1], edges):
            continue
        if not is_point_inside_or_on_boundary(c2[0], c2[1], edges):
            continue
            
        if rect_intersects_edges(x_min, x_max, y_min, y_max, edges):
            continue
            
        return area, p1, p2
        
    return 0, None, None

area, p1, p2 = solve_part2_with_details(input_points)
print(f"Part 2 Max Area: {area}")
print(f"Formed by points: {p1} and {p2}")
